In [10]:
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
#from statsmodels.tsa.holtwinters import ExponentialSmoothing
#import plotly.express as px
from pmdarima import auto_arima
#from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
#from arch import arch_model

In [2]:
'''
con_str = (
    "Driver={SQL Server};"
    "Server=lnmxvmsqlqa02;"
    "Database=HDI_DWH;"
    "Trusted_Connection=yes"
)
'''
url = URL(
    account='xxxxx',
    user='xxxxx',
    password='xxxxx',
    database='xxxx',
    schema='PUBLIC',
    warehouse='DATALAKE_WH',
    role='VENTAS_ROLE'    
)

engine = create_engine(url)
conn = engine.connect()

query = """
SELECT 
        date_from_parts(tecnica.periodo/100, tecnica.periodo%100,01) as fecha,
        oficina.direccion,
        oficina.subdireccion,
        oficina.oficinacomercial "OFICINA",
        tecnica.primanetapropiasincoaseguro as "PrimaEmitida"
        
FROM TB_BI_AUTRFACTEMISIONDOC emision
    INNER JOIN TB_BI_AUTRBASETECNICA  tecnica
        ON emision.numcompletocotizacion = tecnica.numcompletocotizacion
        AND emision.numdocumento = tecnica.numdocumento
    LEFT JOIN TB_BI_CATAGENTE agente
        ON emision.nipperfilagente = agente.nipperfilagente
    LEFT JOIN TB_DL_CATOFICINACOMERCIAL oficina
        ON agente.idoficinaagente = oficina.idoficina
WHERE 
    tecnica.periodo between 202001 and 202505
    and emision.idtipopoliza = 4013
    and direccion in ('Bajio - Occidente','Norte','Mexico - Sur')
    and emision.idtipovehiculo in (4579, 3829)
GROUP BY all;
"""
try:     
    df = pd.read_sql(query,conn)
    print(df.head(10))
    conn.close()
except pyodbc.Error as ex:
    print("Error en SQL Server:", ex)



        fecha          direccion        subdireccion  \
0  2022-11-01       Mexico - Sur  Mexico Promotorias   
1  2022-11-01              Norte             Noreste   
2  2022-11-01       Mexico - Sur                 Sur   
3  2022-11-01  Bajio - Occidente           Occidente   
4  2022-11-01       Mexico - Sur  Mexico Promotorias   
5  2022-01-01  Bajio - Occidente               Bajio   
6  2022-11-01  Bajio - Occidente               Bajio   
7  2022-11-01  Bajio - Occidente               Bajio   
8  2022-11-01              Norte            Noroeste   
9  2022-11-01       Mexico - Sur  Mexico Promotorias   

                          oficina  PrimaEmitida  
0     Promotorías México Satélite        0.0000  
1                         Torreón    -6467.4547  
2                          Cancún        0.0000  
3           Guadalajara Despachos        0.0000  
4  Promotorías México Insurgentes     7620.6303  
5                         Pachuca        0.0000  
6                         Pachuca

In [11]:
#Ajustar datos
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.groupby(['subdireccion','fecha'])['PrimaEmitida'].sum().reset_index()
#df = df.pivot(index='fecha', columns='subdireccion', values='PrimaEmitida').fillna(0)

In [59]:
# Validar formato de fechas
invalid_dates = df[~df['fecha'].apply(lambda x: isinstance(x, pd.Timestamp))]
if not invalid_dates.empty:
    print("Fechas inválidas:\n", invalid_dates)

In [22]:
#scale_factor = 1e5
#df_scale = df / scale_factor
#Forecast
forecasts= []
for subdireccion in df['subdireccion'].unique():
    df_subdir = df[df['subdireccion'] == subdireccion].set_index('fecha').asfreq('MS')['PrimaEmitida']

    if not isinstance(df_subdir.index, pd.DatetimeIndex):
        df_subdir.index = pd.to_datetime(df_subdir.index)

    #modelo SARIMA con autoarima
    model = auto_arima(df_subdir, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)
    print(f"SubDireccion: {subdireccion}, SARIMA: {model.order}, Seasonal: {model.seasonal_order}")

    n_periods= 6
    forecast, conf_int = model.predict(n_periods=n_periods, return_conf_int=True)

    #DF
    future_fechas = pd.date_range(start='2025-05-01', periods=n_periods, freq='MS')
    forecast_df = pd.DataFrame({
        'Fechas':future_fechas,
        'Forecast': forecast,
        'LOWER_CI': conf_int[:,0],
        'UPPER_CI': conf_int[:,1],
        'P_Moncada': conf_int[:,1]*.95,
        'SubDir': subdireccion
    })
    forecasts.append(forecast_df)
    
forecast_fin = pd.concat(forecasts)
   


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=2127.116, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=2128.223, Time=0.03 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=2128.097, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=2125.138, Time=0.01 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=2128.518, Time=0.02 sec
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=2128.596, Time=0.02 sec
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=2130.346, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=2126.870, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=2126.692, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=2123.688, Time=0.03 sec
 ARIMA(1,1,1)(1,0,0)[12] intercept   : AIC=2125.392, Time=0.08 sec
 ARIMA(1,1,1)(0,0,1)[12] intercept   : AIC=2125.149, Time=0.07 sec
 ARIMA(1,1,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(2,1,1)(0,0,0)[12] inte

In [23]:
#Guardar datos en excel
forecasts_final = pd.concat(forecasts)
forecasts_final.to_excel('forecast_prima_2025.xlsx', index = False)
#forecast_final.to_sql('forecast_prima_2025', conn, if_exists='replace', index=False)
print("done")

done
